# 2025 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [1]:
import json
import os
import pandas as pd
import torch

In [2]:
from huggingface_hub import login


login("运行这个之前改成你们自己的token，git不准我上传我的")
                


C:\Users\Tom\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## DATA COLLECTION

In [3]:
import os, json, pandas as pd


DATA_DIR   = "./data"
TRAIN_FILE = os.path.join(DATA_DIR, "train-claims.json")
DEV_FILE   = os.path.join(DATA_DIR, "dev-claims.json")
TEST_FILE  = os.path.join(DATA_DIR, "test-claims-unlabelled.json")
EVID_FILE  = os.path.join(DATA_DIR, "evidence.json")


def load_claims(path: str, labelled: bool = True) -> pd.DataFrame:
   
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    rows = []
    for cid, info in raw.items():
        row = {
            "claim_id":   cid,
            "claim_text": info.get("claim_text", "")
        }
        if labelled:
            row["label"]      = info["claim_label"]
            row["evid_ids"]   = info["evidences"]     # list[str]
        rows.append(row)

    df = pd.DataFrame(rows)

   
    if labelled:
        df["label"] = df["label"].astype("category")

    return df


def load_evidence(path: str):
    
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    df   = pd.DataFrame([{"evid_id": k, "evid_text": v} for k, v in raw.items()])
    edict = {k: v for k, v in raw.items()}
    return df, edict



df_train = load_claims(TRAIN_FILE, labelled=True)
df_dev   = load_claims(DEV_FILE,   labelled=True)
df_test  = load_claims(TEST_FILE,  labelled=False)

df_evid, evid_dict = load_evidence(EVID_FILE)    


LABEL2ID = {"SUPPORTS": 0, "REFUTES": 1, "NOT_ENOUGH_INFO": 2, "DISPUTED": 3}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

for df in (df_train, df_dev):
    df["label_id"] = df["label"].map(LABEL2ID).astype("int8")

print(f"Train size: {len(df_train):,}")
print(f"Dev   size: {len(df_dev):,}")
print(f"Test  size: {len(df_test):,}")
print(f"Evidence passages: {len(df_evid):,}")

display(df_train.head())
display(df_evid.head())

print("Train label distribution:")
display(df_train["label"].value_counts())

print("Dev label distribution:")
display(df_dev["label"].value_counts())


Train size: 1,228
Dev   size: 154
Test  size: 153
Evidence passages: 1,208,827


,claim_id,claim_text,label,evid_ids,label_id
0,claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1...",3
1,claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]",1
2,claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]",0
3,claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5...",3
4,claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72...",2


,evid_id,evid_text
0,evidence-0,"John Bennet Lawes, English entrepreneur and ag..."
1,evidence-1,Lindberg began his professional career at the ...
2,evidence-2,``Boston (Ladies of Cambridge)'' by Vampire We...
3,evidence-3,"Gerald Francis Goyer (born October 20, 1936) w..."
4,evidence-4,He detected abnormalities of oxytocinergic fun...


Train label distribution:


label
SUPPORTS           519
NOT_ENOUGH_INFO    386
REFUTES            199
DISPUTED           124
Name: count, dtype: int64

Dev label distribution:


label
SUPPORTS           68
NOT_ENOUGH_INFO    41
REFUTES            27
DISPUTED           18
Name: count, dtype: int64

## 预处理


In [4]:
!pip install -q rank_bm25
!pip install -q tqdm
!pip install -q sentence-transformers hnswlib
!pip install -U bitsandbytes transformers accelerate

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [5 lines of output]
  running bdist_wheel
  running build
  running build_ext
  building 'hnswlib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for hnswlib
ERROR: Failed to build installable wheels for some pyproject.toml based projects (hnswlib)


In [5]:
CLIMATE_CONCEPTS = {
    "physical_mechanisms": [
        "greenhouse effect", "carbon dioxide", "CO2", "methane", "CH4", 
        "greenhouse gas", "GHG", "emissions", "fossil fuel", "carbon cycle",
        "radiative forcing", "albedo", "feedback", "sensitivity"
    ],
    "observations": [
        "temperature", "warming", "cooling", "precipitation", "sea level", 
        "ice sheet", "glacier", "sea ice", "ocean acidification", "drought",
        "flood", "extreme weather", "heat wave", "storm", "hurricane"
    ],
    "climate_systems": [
        "atmosphere", "ocean", "cryosphere", "biosphere", "El Niño", "La Niña", 
        "jet stream", "gulf stream", "AMOC", "PDO", "AMO", "ENSO", "monsoon"
    ],
    "time_periods": [
        "pre-industrial", "industrial", "holocene", "anthropocene", "pleistocene",
        "ice age", "medieval warm period", "little ice age", "paleoclimate"
    ]
}

In [6]:
import hnswlib



## 粗提取-1



In [48]:


import pathlib, gzip, pickle, json, numpy as np, hnswlib
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from nltk.tokenize import wordpunct_tokenize
K_BM25   = 600    
N_DENSE  = 600    
EF_QUERY = 200    

def tokenize(text: str):
    return [tok.lower() for tok in wordpunct_tokenize(text) if tok.isalnum()]


corpus_tokens = [tokenize(t) for t in evid_dict.values()]
bm25          = BM25Okapi(corpus_tokens)
evid_list     = list(evid_dict.keys())         


def get_bm25_topk(df, K=K_BM25):
    cache_path = pathlib.Path(f"bm25_top{K}.pkl.gz")
    if cache_path.exists():
        with gzip.open(cache_path, "rb") as f:
            print(f"✓ load BM25 cache ({cache_path})")
            return pickle.load(f)

    claim_topk = {}
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"BM25 Top-{K}"):
        idxs = np.argsort(bm25.get_scores(tokenize(row.claim_text)))[::-1][:K]
        claim_topk[row.claim_id] = [evid_list[i] for i in idxs]

    with gzip.open(cache_path, "wb") as f:
        pickle.dump(claim_topk, f)
    print(f"⌛ save cache to {cache_path}")
    return claim_topk

                 
bm25_topk = get_bm25_topk(df_dev, K_BM25)

# MiniLM + HNSW 
MODEL = "all-MiniLM-L6-v2"
minilm = SentenceTransformer(MODEL)
DIM    = minilm.get_sentence_embedding_dimension()

hnsw_bin = pathlib.Path("evidence_hnsw.bin")
ids_npy  = pathlib.Path("evid_ids.npy")
evid_ids = np.load(ids_npy)
if not hnsw_bin.exists():
    
    ev_emb = minilm.encode(list(evid_dict.values()),
                           batch_size=256,
                           normalize_embeddings=True,
                           show_progress_bar=True)
    np.save(ids_npy, np.array(evid_list))

    
    idx = hnswlib.Index(space="cosine", dim=DIM)
    idx.init_index(max_elements=len(ev_emb), ef_construction=200, M=32)
    idx.add_items(ev_emb)
    idx.save_index(hnsw_bin)
else:
    idx = hnswlib.Index(space="cosine", dim=DIM)
    idx.load_index(str(hnsw_bin), max_elements=len(evid_ids))


idx.set_ef(EF_QUERY)                 

def dense_retrieve(text, k=N_DENSE):
    q_vec = minilm.encode(text, normalize_embeddings=True)
    I, _ = idx.knn_query(q_vec, k=k)
    return [evid_ids[i] for i in I[0]]


claim_topk = {}
for _, row in tqdm(df_dev.iterrows(), total=len(df_dev),
                   desc="BM25∪HNSW retrieval"):
    merged = list(dict.fromkeys(
        bm25_topk[row.claim_id] + dense_retrieve(row.claim_text)))
    claim_topk[row.claim_id] = merged



def recall_at_k(dev_df, topk_map):
    rs = []
    for _, r in dev_df.iterrows():
        gold  = set(r.evid_ids)
        found = len(gold & set(topk_map[r.claim_id]))
        rs.append(found / len(gold))
    return sum(rs) / len(rs)

recall_val = recall_at_k(df_dev, claim_topk)
print(f"\nRecall(BM25∪HNSW) on dev: {recall_val:.3f}")

✓ load BM25 cache (bm25_top600.pkl.gz)


BM25∪HNSW retrieval: 100%|██████████| 154/154 [00:01<00:00, 110.43it/s]


Recall(BM25∪HNSW) on dev: 0.913


## 粗提取-2

In [ ]:
# 这个是第一版本，recall大概是0.69

def simple_tokenize(text):
    
    words = re.findall(r'\w+', text.lower())
    return [w for w in words if len(w) > 3] 

def calculate_recall(gold_evidence_ids, predicted_evidence_ids):
   
    if not gold_evidence_ids:
        return 1.0 
    
    found = len(set(gold_evidence_ids) & set(predicted_evidence_ids))
    return found / len(gold_evidence_ids)

def improved_coarse_retrieval(df, evid_dict, bm25, evid_list, minilm, idx, evid_ids, 
                              candidates_per_claim=200, cache_path="improved_coarse_candidates.pkl.gz"):
    
    
    
    cache_file = pathlib.Path(cache_path)
    if cache_file.exists():
        with gzip.open(cache_file, "rb") as f:
            print(f"✓ load cache ({cache_file})")
            cached_data = pickle.load(f)
            return cached_data['candidates'], cached_data['recall']
    
    claim_candidates = {}
    recall_values = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="improved "):
        claim_id = row['claim_id']
        claim_text = row['claim_text']
        
       
        gold_evidence = row.get('evid_ids', [])
        
        
        claim_keywords = set(simple_tokenize(claim_text))
        keyword_candidates = []
        
        if len(claim_keywords) > 0:
            
            sample_size = min(10000, len(evid_dict))
            sample_eids = list(evid_dict.keys())[:sample_size]
            
            for eid in sample_eids:
                
                evid_sample = evid_dict[eid][:200]
                evid_words = set(simple_tokenize(evid_sample))
                overlap = len(claim_keywords & evid_words)
                
               
                if overlap >= min(2, len(claim_keywords) // 3):
                    keyword_candidates.append(eid)
            
            keyword_candidates = keyword_candidates[:500]  
        
        
        try:
            
            bm25_scores = bm25.get_scores([w.lower() for w in re.findall(r'\w+', claim_text) if len(w) > 2])
            top_bm25_indices = np.argsort(bm25_scores)[::-1][:300]
            bm25_candidates = [evid_list[i] for i in top_bm25_indices]
        except Exception as e:
            print(f"BM25 error: {e}")
            bm25_candidates = []
        
        
        try:
            q_embedding = minilm.encode(claim_text, normalize_embeddings=True)
            I, _ = idx.knn_query(q_embedding.reshape(1, -1), k=300)
            dense_candidates = [evid_ids[i] for i in I[0]]
        except Exception as e:
            print(f"dense error: {e}")
            dense_candidates = []
        
        
        merged_candidates = []
        
        
        bm25_set = set(bm25_candidates[:100])
        dense_set = set(dense_candidates[:100])
        high_confidence = list(bm25_set.intersection(dense_set))
        merged_candidates.extend(high_confidence)
        
        
        for i in range(min(75, len(bm25_candidates))):
            if bm25_candidates[i] not in merged_candidates:
                merged_candidates.append(bm25_candidates[i])
        
        for i in range(min(75, len(dense_candidates))):
            if dense_candidates[i] not in merged_candidates:
                merged_candidates.append(dense_candidates[i])
        
        
        if keyword_candidates:
            keyword_set = set(keyword_candidates)
            for candidate in list(keyword_set - set(merged_candidates))[:50]:
                merged_candidates.append(candidate)
        
        
        remaining = list(set(bm25_candidates + dense_candidates) - set(merged_candidates))
        merged_candidates.extend(remaining[:max(0, candidates_per_claim - len(merged_candidates))])
        
        
        final_candidates = merged_candidates[:candidates_per_claim]
        claim_candidates[claim_id] = final_candidates
        
        
        if hasattr(row, 'evid_ids') and row.evid_ids:
            recall = calculate_recall(row.evid_ids, final_candidates)
            recall_values.append(recall)
    
    
    avg_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    
    
    with gzip.open(cache_file, "wb") as f:
        cache_data = {
            'candidates': claim_candidates,
            'recall': avg_recall
        }
        pickle.dump(cache_data, f)
    print(f"⌛ save cache {cache_file}")
    
    print(f"\naverage recall: {avg_recall:.4f}")
    print(f"candicates num: {candidates_per_claim}")
    
    
    if len(recall_values) > 0:
        
        for i, (_, row) in enumerate(df.iterrows()):
            if hasattr(row, 'evid_ids') and i < 5: 
                cid = row['claim_id']
                gold_count = len(row.evid_ids)
                found = len(set(row.evid_ids) & set(claim_candidates[cid]))
                
    
    return claim_candidates, avg_recall


improved_candidates, improved_recall = improved_coarse_retrieval(
    df_dev,
    evid_dict, 
    bm25, 
    evid_list, 
    minilm, 
    idx, 
    evid_ids,
    candidates_per_claim=200 
)

print(f"\n recall: {improved_recall:.4f}")



In [46]:
# 这个是改进的粗提取模块2.0版本
from tqdm.auto import tqdm
import re
import numpy as np
import pathlib, gzip, pickle

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import hnswlib
from sklearn.metrics.pairwise import cosine_similarity

def simple_tokenize(text):
    
    words = re.findall(r'\w+', text.lower())
    return [w for w in words if len(w) > 3]

def calculate_recall(gold_ids, pred_ids):
    
    if not gold_ids:
        return 1.0
    return len(set(gold_ids) & set(pred_ids)) / len(gold_ids)

def improved_coarse_retrieval(df,
                              evid_dict,
                              bm25: BM25Okapi,
                              evid_list,
                              sbert: SentenceTransformer,
                              hnsw_index: hnswlib.Index,
                              evid_ids,
                              k_bm25=100,
                              k_dense=100,
                              rrf_k=60,
                              mmr_lambda=0.7,
                              candidates_per_claim=200,
                              cache_path="improved_coarse_candidates.pkl.gz"):

    cache_file = pathlib.Path(cache_path)
    if cache_file.exists():
        with gzip.open(cache_file, "rb") as f:
            data = pickle.load(f)
        print(f"✓ load from cache: {cache_file}")
        return data['candidates'], data['recall']
    
    claim_candidates = {}
    recall_vals = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="improved"):
        cid = row['claim_id']
        text = row['claim_text']
        gold = row.get('evid_ids', [])
        
        # 1. BM25
        tokens = simple_tokenize(text)
        bm25_scores = bm25.get_scores(tokens)
        top_bm25 = np.argsort(-bm25_scores)[:k_bm25]
        bm25_ids = [evid_list[i] for i in top_bm25]
        bm25_rank = {evid_list[i]: r for r,i in enumerate(top_bm25)}
        
        # 2. Dense 
        q_emb = sbert.encode(text, normalize_embeddings=True)
        labels, _ = hnsw_index.knn_query(q_emb.reshape(1, -1), k=k_dense)
        dense_idx = labels[0]
        dense_ids = [evid_ids[i] for i in dense_idx]
        dense_rank = {evid_ids[i]: r for r,i in enumerate(dense_idx)}
        
        # 3. RRF
        fusion = {}
        for eid in set(bm25_ids + dense_ids):
            score = 0.0
            if eid in bm25_rank:
                score += 1/(bm25_rank[eid] + rrf_k)
            if eid in dense_rank:
                score += 1/(dense_rank[eid] + rrf_k)
            fusion[eid] = score
        fused = [eid for eid,_ in sorted(fusion.items(), key=lambda x: -x[1])]
        
        # 4. MMR
        pool = fused[:2*candidates_per_claim]
        pool_embs = sbert.encode([evid_dict[e] for e in pool], normalize_embeddings=True)
        sim_q = cosine_similarity(q_emb.reshape(1,-1), pool_embs)[0]
        
        selected, cand_ids, cand_embs = [], pool.copy(), pool_embs.copy()
        sim_to_q = sim_q.copy()
        while len(selected) < candidates_per_claim and cand_ids:
            if not selected:
                idx0 = int(np.argmax(sim_to_q))
            else:
                sims = cosine_similarity(q_emb.reshape(1,-1), cand_embs)[0]
                sel_embs = sbert.encode([evid_dict[e] for e in selected], normalize_embeddings=True)
                sim_sel = cosine_similarity(cand_embs, sel_embs)
                max_sel = np.max(sim_sel, axis=1)
                mmr_scores = mmr_lambda * sims - (1-mmr_lambda) * max_sel
                idx0 = int(np.argmax(mmr_scores))
            selected.append(cand_ids.pop(idx0))
            cand_embs = np.delete(cand_embs, idx0, axis=0)
            sim_to_q = np.delete(sim_to_q, idx0)
        
        claim_candidates[cid] = selected
        recall_vals.append(calculate_recall(gold, selected))
    
    mean_rec = float(np.mean(recall_vals))
    
    with gzip.open(cache_file, "wb") as f:
        pickle.dump({'candidates': claim_candidates, 'recall': mean_rec}, f)
    print(f"⌛ cache: {cache_file}")
    print(f"average recall : {mean_rec:.4f}  |  candicate: {candidates_per_claim}")
    
   
    print("\n sample:")
    for i, row in df.head(3).iterrows():
        cid = row['claim_id']
        gold = row.get('evid_ids', [])
        found = len(set(gold)&set(claim_candidates[cid]))
        print(f"{cid}: {found}/{len(gold)} = {found/len(gold):.2f}")
    
    return claim_candidates, mean_rec


improved_cands, improved_rec = improved_coarse_retrieval(
    df_dev, evid_dict, bm25, evid_list, minilm, idx, evid_ids,
    k_bm25=100, k_dense=100, rrf_k=60,
    mmr_lambda=0.7, candidates_per_claim=200
)
print(f"\nfinal Recall: {improved_rec:.4f}")


✓ load from cache: improved_coarse_candidates.pkl.gz

final Recall: 0.7158


# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
if torch.cuda.is_available():
    print("CUDA is available.")

    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

    print(f"Current GPU device: {torch.cuda.current_device()}")
else:
    print("CUDA is not available. Running on CPU.")

CUDA is available.
Number of available GPUs: 1
GPU 0: NVIDIA GeForce RTX 4070 SUPER
Current GPU device: 0


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*